In [6]:
import pandas as pd 
import numpy as np

from difflib import get_close_matches

import re

In [7]:
df_final = pd.read_csv('../../Data/final_movies.csv')
df_budget= pd.read_csv('../../Data/movie_budgets_name_fixed (2).csv')

In [9]:
def reformat_title(title):
    formated_str = re.sub(r"[^a-zA-Z0-9:' .]", '', title).strip()
    formated_str = re.sub(' +', ' ', formated_str)
    return formated_str.replace(' ','').lower()

In [10]:
df_final['Reformated name'] = df_final['Name'].apply(reformat_title)
df_budget['Reformated name'] = df_budget['Movie Name'].apply(reformat_title)

In [11]:
count = 0

title_list = df_final['Reformated name']

for i in range(len(df_budget['Reformated name'])):
    match = get_close_matches(df_budget['Reformated name'][i], title_list, n = 1, cutoff=0.9)
    if len(match) > 0:
        df_budget.loc[i, 'Reformated name'] = match[0]
        count += 1

print('\n',count)


 5230


In [13]:
def extract_year(year_str):
    if isinstance(year_str, str):
        match = re.search(r'\b(19|20)\d{2}\b', year_str)
        if match:
            return match.group()
        match = re.search(r'(19|20)\d{2}', year_str)
        if match:
            return match.group()
        if year_str.lower() == 'unknown' or year_str.lower() == 'nan':
            return np.nan
    return np.nan 

In [14]:
df_final['Year'] = df_final['Year'].apply(extract_year)
df_budget['Year'] = df_budget['Release Date'].apply(extract_year)

In [15]:
df_fulldata = pd.merge(df_final, df_budget, on=['Reformated name'], how='inner')
df_fulldata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5257 entries, 0 to 5256
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Name               5257 non-null   object
 1   Year_x             5195 non-null   object
 2   CriticScore        4242 non-null   object
 3   UserScore          4646 non-null   object
 4   Link               5257 non-null   object
 5   PlatformReleased   5257 non-null   object
 6   Cast               5224 non-null   object
 7   Director           5220 non-null   object
 8   Genre              5210 non-null   object
 9   Rating             4118 non-null   object
 10  Runtime            5183 non-null   object
 11  Studio             5209 non-null   object
 12  Reformated name    5257 non-null   object
 13  Release Date       5257 non-null   object
 14  Movie Name         5257 non-null   object
 15  Production Budget  5257 non-null   object
 16  Domestic Gross     5257 non-null   object


In [16]:
df_fulldata.rename(columns={'Name' : 'Title'}, inplace=True)
df_fulldata = df_fulldata.drop('Year_x', axis=1)
df_fulldata = df_fulldata.drop('Year_y', axis=1)
df_fulldata = df_fulldata.drop('Movie Name', axis=1)
df_fulldata = df_fulldata.drop('Reformated name', axis=1)

df_fulldata.drop_duplicates(subset='Title', ignore_index=True, inplace=True)

df_fulldata.to_csv('../../Data/full_data.csv', index=False)

In [17]:
df_result_1 = pd.read_csv('../../Data/results.csv')
df_result_2 = pd.read_csv('../../Data/updated_output.csv')

In [18]:
df_result_2 = df_result_2.drop(columns='Identifier', axis = 1)
df_result_2.rename(columns={'Movie Title' : 'Title'}, inplace=True)
df_result_2.rename(columns={'metascore' : 'Metascore'}, inplace=True)
df_result_2.rename(columns={'userscore' : 'Userscore'}, inplace=True)


In [19]:
df_result = pd.concat([df_result_1, df_result_2], ignore_index=True, sort=False)
df_result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5379 entries, 0 to 5378
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Title      5379 non-null   object
 1   Metascore  4742 non-null   object
 2   Userscore  4742 non-null   object
dtypes: object(3)
memory usage: 126.2+ KB


In [20]:
df_result['Formated name'] = df_result['Title'].apply(reformat_title)
df_fulldata['Formated name'] = df_fulldata['Title'].apply(reformat_title)
df_result['Formated name'] = df_result['Title'].apply(reformat_title)

In [21]:
count = 0
count = 0

title_list = df_fulldata['Formated name']

for i in range(len(df_result['Formated name'])):
    match = get_close_matches(df_result['Formated name'][i], title_list, n = 1, cutoff=0.9)
    if len(match) > 0:
        df_result.loc[i, 'Formated name'] = match[0]
        count += 1

print('\n',count)


 5377


In [22]:
df_fulldata

,Title,CriticScore,UserScore,Link,PlatformReleased,Cast,Director,Genre,Rating,Runtime,Studio,Release Date,Production Budget,Domestic Gross,Worldwide Gross,Formated name
0,L.A. Confidential,99%,94%,https://www.rottentomatoes.com/m/la_confidential,Cinema,"Kevin Spacey, Russell Crowe, Guy Pearce, James...",Curtis Hanson,"Crime, Drama",R,2h 16m,Warner Home Vídeo,"Sep 19, 1997","$35,000,000","$64,604,977","$126,204,977",l.a.confidential
1,The Godfather,97%,98%,https://www.rottentomatoes.com/m/the_godfather,Cinema,"Marlon Brando, Al Pacino, James Caan, Richard ...",Francis Ford Coppola,"Crime, Drama",R,2h 57m,Paramount Pictures,"Mar 15, 1972","$7,000,000","$136,479,994","$270,007,394",thegodfather
2,Casablanca,99%,95%,https://www.rottentomatoes.com/m/1003707-casab...,Cinema,"Humphrey Bogart, Ingrid Bergman, Paul Henreid,...",Michael Curtiz,Drama,PG,1h 42m,Warner Bros. Pictures,"Jan 23, 1943","$1,039,000","$10,462,500","$10,567,179",casablanca
3,Parasite,99%,90%,https://www.rottentomatoes.com/m/parasite_2019,Cinema,"Song Kang-ho, Lee Sun-kyun, Jo Yeo-jeong, Choi...",Bong Joon Ho,"Comedy, Mystery & Thriller, Drama",R,2h 12m,Neon,"May 30, 2019","$11,800,000","$53,369,745","$253,267,858",parasite
4,Top Gun: Maverick,96%,99%,https://www.rottentomatoes.com/m/top_gun_maverick,Cinema,"Tom Cruise, Miles Teller, Jennifer Connelly, J...",Joseph Kosinski,"Action, Adventure",PG-13,2h 11m,Paramount Pictures,"May 20, 2022","$170,000,000","$718,732,821","$1,464,400,233",topgun:maverick
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5114,Antarctic Edge: 70 South,57%,50%,https://www.rottentomatoes.com/m/antarctic_edg...,Cinema,"Dena Seidel, Richard Ludescher, Dena Seidel",Dena Seidel,"Documentary, Adventure",NaN,1h 12m,First Run,"Apr 17, 2015","$150,000","$7,193","$7,193",antarcticedge:70south
5115,8 Days to Hell,NaN,NaN,https://www.rottentomatoes.com/m/8_days_to_hell,Cinema,"Eric Roberts, Shane Woodson, Drew Hale, David ...",Shane Woodson,Horror,NaN,1h 25m,Pegasus Flying Films,"Jul 30, 2021","$100,000",$0,$0,8daystohell
5116,20 Dates,35%,47%,https://www.rottentomatoes.com/m/20_dates,Cinema,"Myles Berkowitz, Elisabeth Wagner, Richard Arl...",Myles Berkowitz,"Comedy, Romance",R,1h 32m,Fox,"Feb 26, 1999","$66,000","$541,636","$602,920",20dates
5117,Happy 40th,NaN,NaN,https://www.rottentomatoes.com/m/happy_40th,Cinema,"Fernando Acosta, Jenni Blong, Robyn Cohen, Mad...",Madoka Raine,Drama,NaN,1h 40m,English,Unknown,"$35,000",$0,$0,happy40th


In [23]:
df_fulldata = pd.merge(df_fulldata, df_result, on=['Formated name'], how='inner')
df_fulldata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5399 entries, 0 to 5398
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Title_x            5399 non-null   object
 1   CriticScore        4373 non-null   object
 2   UserScore          4784 non-null   object
 3   Link               5399 non-null   object
 4   PlatformReleased   5399 non-null   object
 5   Cast               5366 non-null   object
 6   Director           5361 non-null   object
 7   Genre              5352 non-null   object
 8   Rating             4244 non-null   object
 9   Runtime            5326 non-null   object
 10  Studio             5351 non-null   object
 11  Release Date       5399 non-null   object
 12  Production Budget  5399 non-null   object
 13  Domestic Gross     5399 non-null   object
 14  Worldwide Gross    5399 non-null   object
 15  Formated name      5399 non-null   object
 16  Title_y            5399 non-null   object


In [24]:
df_fulldata = df_fulldata.drop(columns='Title_y', axis=1)
df_fulldata.rename(columns={'Title_x' : 'Title'}, inplace=True)
df_fulldata.drop_duplicates(subset='Title', ignore_index=True, inplace=True)

In [25]:
df_fulldata.to_csv("../../Data/full_data.csv", index= False)